In [5]:
pip install tensorflow

  Using cached protobuf-4.23.3-cp39-cp39-win_amd64.whl (422 kB)
  Using cached tensorflow_estimator-2.12.0-py2.py3-none-any.whl (440 kB)
  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
  Using cached tensorboard_data_server-0.7.1-py3-none-any.whl (2.4 kB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.10.0
    Uninstalling tensorflow-estimator-2.10.0:
      Successfully uninstalled tensorflow-estimator-2.10.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.6.1
    Uninstalling tensorboard-data-server-0.6.1:
      Successfully uninstalled tensorboard-data-server-0.6.1
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
  Attempting uninstall: keras
    Found existing installation: keras 2.10.0
    Uninstalling keras-2.10.0:
      Successfully uninstalled ke

In [3]:
pip install --upgrade pip

     ---------------------------------------- 2.1/2.1 MB 11.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [33]:
pip install keras_preprocessing

Note: you may need to restart the kernel to use updated packages.
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)


In [39]:
import tensorflow as tf
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler,\
SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from transformers import AdamW, BertForSequenceClassification, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### gpu 설정

In [24]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0' : 
    raise SystemError('GPU device not found')
print(f'Found GPU at: {device_name}')

SystemError: GPU device not found

In [11]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4211019393632157731
xla_global_id: -1
]


### CUDA를 Torch용 디바이스로 지정하기

In [40]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

AssertionError: Torch not compiled with CUDA enabled

### 데이터셋 로드하기

In [41]:
#@title Loading the Dataset
#source of dataset : https://nyu-mll.github.io/CoLA/
df = pd.read_csv('./data/in_domain_train.tsv', delimiter='\t', header=None,
                 names=['sentence_source', 'label', 'label_notes', 'sentence'])
df.shape

(8551, 4)

In [43]:
df.sample(10)

,sentence_source,label,label_notes,sentence
1645,r-67,1,NaN,i brought john a razor to shave himself with .
2530,l-93,0,?*,jennifer craned her arm .
2581,l-93,0,*,water pours easily onto the plants .
2055,rhl07,1,NaN,"one of the jewish children is a spunky girl , ..."
4343,ks08,0,*,he believed there to be likely that he won the...
4020,ks08,1,NaN,the foxes seem compatible with the chickens .
5093,ks08,1,NaN,insensitive is how i would describe him .
8511,ad03,0,*,peter is those pigs .
2559,l-93,0,*,i funneled the mixture to rina .
471,bc01,1,NaN,i believed these students all to like john .
